In [11]:
device = torch.device("cuda")

In [2]:
params = GenerateParams(decoding_method="sample", max_new_tokens=1024, temperature = 1, top_p = 1, top_k = 50, typical_p = 1, stop_sequences=["\nQ: ","Use just the given patient history to answer the question."], return_options={"generated_tokens" : True, "token_logprobs" : True})
greedy_params = GenerateParams(decoding_method="greedy", max_new_tokens=1024, repetition_penalty=1.1, stop_sequences=["\nQ: ","Use just the given patient history to answer the question."], return_options={"generated_tokens" : True, "token_logprobs" : True})
# Instantiate a model proxy object to send your requests
model = Model("meta-llama/Llama-2-7b-chat", params=params, credentials=creds)
greedy_model = Model("meta-llama/Llama-2-7b-chat", params=greedy_params, credentials=creds)

reasoning_delimiter = "Answer: "
output_delimiter = "Q:"
instruction = "Use just the given patient history to answer the question. Do not assume any further information about the patient. Strictly Limit your response to 200 words."

In [3]:
def option_exists(new_op,old_ops):
    new_op = new_op.strip(". ").lower()
    for old_op in old_ops:
        old_op = old_op.strip(". ").lower()
        # print(f"new:{new_op}")
        # print(f"old:{old_op}")
        if (new_op in old_op or old_op in new_op):
            return(1)
    return(0)

def create_options(instring, num_unique_ops = 4, options_generate_limit = 15):
    unique_options = []
    all_options = []
    op_to_reason = {}

    # inputs = tokenizer(instring, return_tensors="pt")
    # inputs = inputs.to(device)

    print("Creating Options")

    while(len(all_options) < options_generate_limit and len(unique_options) < num_unique_ops):

        if not len(all_options):
            outputs = greedy_model.generate([instring])[0]
            # outputs = model.generate(inputs.input_ids, max_new_tokens = 1024, repetition_penalty = 1.1)
        else:
            outputs = model.generate([instring])[0]
            # outputs = model.generate(inputs.input_ids, max_new_tokens = 1024, do_sample = True, temperature = 1, repetition_penalty = 1.1)

        text_output = outputs.generated_text.strip()
        # text_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        # text_output = text_output.split(instring)[-1]
        # text_output = text_output.strip()

        if reasoning_delimiter in text_output:
            op_reasoning, op_text  = text_output.split(reasoning_delimiter)
            op_reasoning, op_text = op_reasoning.strip(), op_text.strip().split("\n")[0].strip()

            print("Option Created :",op_text)

            if not option_exists(op_text,unique_options):
                print("New Option :", op_text)
                unique_options.append(op_text)
                op_to_reason[op_text] = op_reasoning
            else:
                print("Option already exists.. Discarding..")
            
        else:
            op_text = "<parsing error>"
            print(op_text)

        all_options.append(op_text)
        print("Existing Options :", unique_options, "\n")

    print("Final Options :", unique_options)

    return [unique_options,op_to_reason]


In [8]:
close = False

# st.title("Choose the algorithm you want to run :")
option_ind = int(input("choose an algorithm to run"))
algorithms = ["MedCodex - Greedy", "Codex - Greedy", "MedCodex + Codex (F+B)", "MedCodex + Verifier (F + RM)"]
algo = algorithms[option_ind]
# Drop-down menu with four options
# option = st.selectbox("Choose an option:", ["MedCodex - Greedy", "Codex - Greedy", "MedCodex + Codex (F+B)", "MedCodex + Verifier (F + RM)"])

# Display the selected optio
# st.write("You selected:", option)
print("You selected:", algo)

# user_input = st.text_input("Enter input prompt", "default_value_goes_here")
user_input = input("User input here : ")
print("The input is :\n",user_input)

output = ""

choose an algorithm to run 3


You selected: MedCodex + Verifier (F + RM)


User input here :  A 68-year-old male comes to the physician for evaluation of right flank pain. He has a history of diabetes and peripheral artery disease. His blood pressure is 160/90 mm Hg. Physical examination shows abdominal tenderness and right flank tenderness. An ultrasound shows dilation of the right ureter and renal pelvis.


The input is :
 A 68-year-old male comes to the physician for evaluation of right flank pain. He has a history of diabetes and peripheral artery disease. His blood pressure is 160/90 mm Hg. Physical examination shows abdominal tenderness and right flank tenderness. An ultrasound shows dilation of the right ureter and renal pelvis.


In [28]:
if algo == "MedCodex - Greedy":
    with open("./prompts/kj_instruct_2.txt", 'r') as f:
        few_shot_cot = f.read()

    instruction = "Use just the given patient history to answer the question. Do not assume any further information about the patient. Strictly Limit your response to 200 words."
    instring = f'''{few_shot_cot}\n\n{instruction}\nQ: {user_input}\nA: Let's think step-by-step.'''

elif algo == "Codex - Greedy":
    with open("./prompts/codex_2.txt", 'r') as f:
        few_shot_cot = f.read()
    
    instring = f'''{few_shot_cot}\n\nQ: {user_input}\nA: Let's think step-by-step.'''

reasoning_delimiter = "Answer: "

# submit = st.button("Submit input")
submit = True

if submit:

    # st.write("The input is :\n",user_input)

    # inputs = tokenizer(instring, return_tensors="pt")
    # inputs = inputs.to(device)
    
    greedy_out = greedy_model.generate([instring])[0]
    greedy_output = greedy_out.generated_text.strip()
    # greedy_out = model.generate(inputs.input_ids, max_new_tokens = 1024, repetition_penalty = 1.1)
    # greedy_out = greedy_out.split(instring)[-1]
    # greedy_output = greedy_out.strip()

    # if reasoning_delimiter in greedy_output:
    #     greedy_ans = greedy_output.split(reasoning_delimiter)[1].strip()
    # else:
    #     greedy_ans = "<parsing error>"

    output = greedy_output
    # output = "F output"

The input is :
 A 39-year-old man presents to the emergency department because of progressively worsening chest pain and nausea that started at a local bar 30 minutes prior. The pain radiates to the epigastric area. He has a 5-year history of untreated hypertension. He has smoked 1 pack of cigarettes daily for the past 5 years and started abusing cocaine 2 weeks before his emergency room visit. The patient is diaphoretic and in marked distress. Describe the steps of management for this patient


In [13]:
reward_model = AutoModelForSequenceClassification.from_pretrained('meta-llama/Llama-2-7b-chat-hf', num_labels = 1)
reward_model.load_adapter(f"/dccstor/ojasgr/scripts/approaches/reasoning_verifier/trl/examples/scripts/output_old/7b-chat/checkpoint-1900/")
reward_model.config.pad_token_id = reward_model.config.eos_token_id
reward_model = reward_model.to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-2-7b-chat-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /dccstor/cgdial/ojasgramo/anaconda3/envs/rag/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 114
CUDA SETUP: Loading binary /dccstor/cgdial/ojasgramo/anaconda3/envs/rag/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda114.so...


/dccstor/cgdial/ojasgramo/anaconda3/envs/rag/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/dccstor/cgdial/ojasgramo/anaconda3/envs/rag/lib/libcudart.so.11.0'), PosixPath('/dccstor/cgdial/ojasgramo/anaconda3/envs/rag/lib/libcudart.so')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


In [21]:
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-chat-hf')
tokenizer.pad_token = tokenizer.eos_token

In [27]:
if algo == "MedCodex + Codex (F+B)" or algo == "MedCodex + Verifier (F + RM)":

    with open("./prompts/kj_instruct_2.txt", 'r') as f:
        few_shot_cot = f.read()

    with open("./prompts/codex_2.txt", 'r') as f:
        backward_prompt = f.read()

    instruction = "Use just the given patient history to answer the question. Do not assume any further information about the patient. Strictly Limit your response to 200 words."
    f_instring = f'''{few_shot_cot}\n\n{instruction}\nQ: {user_input}\nA: Let's think step-by-step.'''
    
    uniq_options, op_to_reason = create_options(f_instring)
    if (len(uniq_options) < 4):
        # output = "Not enough options generated!"
        # st.write("Not enough Options")
        print("Not enough Options")

    else:
        if algo == "MedCodex + Codex (F+B)":

            options_text = ""
            for op_num,op in enumerate(uniq_options):
                options_text += f"({chr(ord('A') + op_num)}) {op} "

            options_text = options_text.strip() + '\n'

            backward_ques = input("Input backward Question : ")
            b_instring = f"{backward_prompt}\n\nQ: {backward_ques}\n{options_text}A: Let's think step-by-step."

            # b_inputs = tokenizer(b_instring, return_tensors="pt")
            # b_inputs = b_inputs.to(device)

            b_out = model.generate([b_instring])[0]
            b_output = b_out.generated_text.strip()

            # b_output = model.generate(b_inputs.input_ids, max_new_tokens = 1024, repetition_penalty = 1.1)
            # b_output = b_output.split(b_instring)[-1]
            # b_output = b_output.strip()

            if b_output.strip() == "":
                # st.write("output empty")
                print("output empty")
                b_output = "<empty>"
            
            # if str(elem["a"]).strip() == "":
            #     elem["a"] = "<empty>"
            
            b_answer = re.findall(r"\([A-D]\)",b_output)
            if len(b_answer):
                b_answer = b_answer[0]
                option = ord(b_answer[1]) - ord('A') + 1
                b_answer_idx = ord(b_answer[1]) - ord('A')
                output = b_answer + " " + op_to_reason[uniq_options[b_answer_idx]] + "\nAnswer: " + uniq_options[b_answer_idx]
            else:
                b_answer ="<parsing error>" 
                option = 0
                output = "<parsing error>"


        elif algo == "MedCodex + Verifier (F + RM)":

            instrings = []
            logits = []

            for k in range(4):
                instring = f"Question : {user_input}\nReasoning : Let's think step by step. {op_to_reason[uniq_options[k]]}\nAnswer : {uniq_options[k]}"
                
            inputs = tokenizer(instring, padding = True, return_tensors="pt")
            
            inputs = inputs.to(device)
            with torch.no_grad():
                outputs = reward_model(**inputs)

            # logit = outputs.logits.item()
            # logits.append(logit)

            logits = outputs.logits
            logits = logits.reshape(-1)
            logits = logits.tolist()

            # print('logit', chr(ord('A') + k-1), f"model : {logit:.3f}")

            answer_idx = np.argmax(logits)
            answer = f"({chr(answer_idx + ord('A'))})"

            output = answer + " " + op_to_reason[uniq_options[answer_idx]] + "\nAnswer: " + uniq_options[answer_idx]

Creating Options
Option Created : ordering a CT scan of the abdomen and pelvis.
New Option : ordering a CT scan of the abdomen and pelvis.
Existing Options : ['ordering a CT scan of the abdomen and pelvis.'] 

Option Created : Ordering a urinalysis and urine culture, and performing a plain abdominal X-ray or CT scan.
New Option : Ordering a urinalysis and urine culture, and performing a plain abdominal X-ray or CT scan.
Existing Options : ['ordering a CT scan of the abdomen and pelvis.', 'Ordering a urinalysis and urine culture, and performing a plain abdominal X-ray or CT scan.'] 

Option Created : CT angiogram.
New Option : CT angiogram.
Existing Options : ['ordering a CT scan of the abdomen and pelvis.', 'Ordering a urinalysis and urine culture, and performing a plain abdominal X-ray or CT scan.', 'CT angiogram.'] 

Option Created : Nephrolithiasis.
New Option : Nephrolithiasis.
Existing Options : ['ordering a CT scan of the abdomen and pelvis.', 'Ordering a urinalysis and urine cul

In [28]:
print(output)

(A) The patient has a history of diabetes and peripheral artery disease. The blood pressure is elevated which could indicate hypertension. The physical examination shows abdominal tenderness and right flank tenderness which could indicate kidney stones or infection. The ultrasound shows dilation of the right ureter and renal pelvis which confirms the diagnosis of kidney stones. The patient needs to undergo a CT scan of the abdomen and pelvis to confirm the diagnosis and rule out any complications such as obstruction or infection.
Answer: ordering a CT scan of the abdomen and pelvis.


In [37]:
x = "this is te\n\n next line"
x.strip().split("\n")[0]

'this is te'